# Case 1. Heart Disease Classification

#### Joonas ???, Przemyslaw Zuchmanski
##### 31.01.2020
### Helsinki Metropolia University of Applied Sciences

The main object is to created and train a dense neural network to predict the presence of heart disease on the base of heart disease cleveland data downloaded from the site: https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/ .

##### Data
The data contains values of various health factors usefull in detecting heart diseases. There are 13 factors described in 13 coluns. 14th column describes if the patint sufers from heart disease. The number of records is 303. Missing values (detected in 6 raws) were replaced with 0. 

In [8]:
%pylab inline
import pandas as pd
import numpy
from sklearn import preprocessing
import tensorflow as tf
from sklearn.model_selection import train_test_split


#names of columns
names = ["age","sex","cp","trestbps","chol","fbs","restecg",
                            "thalach","examg","oldpeak","slope","ca","thal","num"]
                   
#reading data and giving names for columns, detecting NaN values
df = pd.read_csv("processed.cleveland.data", 
                 names=names,
                     header=None, 
                     index_col = None, 
                     na_values = '?')

#replacing NaN values with 0
df = df.replace(numpy.NaN,0)

Populating the interactive namespace from numpy and matplotlib


Basic statistics are as follows

In [11]:
print('shape of data set: ', df.shape)
df.describe()

shape of data set:  (303, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,examg,oldpeak,slope,ca,thal,num
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.438944,0.679868,3.158416,131.689769,246.693069,0.148515,0.990099,149.607261,0.326733,1.039604,1.600660,0.663366,4.702970,0.937294
std,9.038662,0.467299,0.960126,17.599748,51.776918,0.356198,0.994971,22.875003,0.469794,1.161075,0.616226,0.934375,1.971038,1.228536
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,48.000000,0.000000,3.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
50%,56.000000,1.000000,3.000000,130.000000,241.000000,0.000000,1.000000,153.000000,0.000000,0.800000,2.000000,0.000000,3.000000,0.000000
75%,61.000000,1.000000,4.000000,140.000000,275.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,1.000000,7.000000,2.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000,7.000000,4.000000


Division data into two subsets: <br>
data - all health factors <br>
labels - indicate if the person is rather sick (1) or healthy (0)

In [12]:
#dividing set to data and labels
data = df.drop(['num'], axis=1)
#converting labels to binary atribut
label = 1.0*(df['num'] >0)

#standarisation
stand_df = preprocessing.scale(df)
#going back to DataFrame an d giving names
df = pd.DataFrame(stand_df)
df.columns = names

##### Models and training
Dividing data. For training and validating we use 80% of samples. Remains 20% we will use for testing.

In [19]:
train_data, test_data, train_labels, test_labels = train_test_split(
                    data,
                    label,
                    test_size = 0.2,
                    random_state = 39,
                    shuffle = True)

### Jonas!! help, What is further i don't know what to do
Creating a model. I have no idea why those factors. if you know plese write here, or you can echcange paramiters for that you know

In [15]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(12, activation='relu', input_shape=(13,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation ='sigmoid')
        ])
model.compile(optimizer = 'adam',
             loss='binary_crossentropy',
             metrics=['Accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Fitting the model. We use 20% of remaining data for validation.

In [23]:
history = model.fit(np.asarray(train_data), np.asarray(train_labels), validation_split=0.2, epochs=5, verbose = 0)
#eval = model.evaluate(np.asarray(test_data), np.asarray(test_labels), verbose=2)
#print(model.metrics_names,': \n', eval)


AttributeError: 'History' object has no attribute 'callbacks'